In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

TO DO:
- Dataset Loading/Choosing ✅
- Dataset Cleaning
- Exploratory Data Analysis
- Baseline (TBD)
- BERT fine-tuning to classify text
- Error Analysis / Robustness Testing

# Dataset Loading/Choosing

- LLM - Detect AI Generated Text Dataset (28k essays)
https://www.kaggle.com/datasets/sunilthite/llm-detect-ai-generated-text-dataset.
- Dataset Card for AI Text Dectection Pile (1.4mil essays)
https://huggingface.co/datasets/artem9k/ai-text-detection-pile


# Dataset Cleaning

- Lowercasing (optional with BERT since it's often case-aware depending on the model)
- Removing HTML tags, extra spaces
- Filtering by length (exclude very short texts)
- Removing duplicates
- Language detection if you need only English
- Where possible tag which model AI text is from

Possible extra feature engineering, usefullness unsure.
BERT embeddings + handcrafted features like:
- Average sentence length
- Readability scores (Flesch-Kincaid)
- N-gram repetition
- Ratio of stopwords Then feed that into a LightGBM/XGBoost model to compare.



# Exploratory Data Analysis (EDA)
- Text length distributions
- Vocabulary richness (e.g. unique words)
- POS tag distribution (maybe AI uses more nouns, fewer adjectives?)
- Visualizations: word clouds, frequency plots
- Clustering to check for seperability of classes

# Baseline (TBD)

- Basic baseline logistic regression etc (Might not be relevant)
- Basic Deep learning artitecture
- Base BERT
- Maybe a basic baseline and a base BERT to see how much performance BERT adds and how much fine-tuned BERT additionally adds.

# BERT fine-tuning to classify text

- BERT vs RoBERTa vs DistilBERT
- RoBERTa often performs better in classification tasks

# Error Analysis / Robustness Testing

- What types of errors does it make confusion matrix?
- Is the model biased toward longer/shorter texts?
- Attention analysis (using tools like BertViz)
- Check if BERT overfits to text length or formatting
- Does it misclassify texts on certain topics?
- Could it unfairly flag texts written by non-native speakers?
- Does it perform better on specific outpurs from specific models?
- Small edits (punctuation, synonyms) and how does they affect the model?
- Test synonym replacements (e.g., "happy" → "joyful") with slight paraphrasing

# Explainability

- Attention Heatmap (with bertviz or transformers-interpret)
- Visualize token importance
- SHAP map

# (Option extra if time allows) Own trained text generator, compare its outputs predictions to the ones from other models